In [1]:
import os
import numpy as np 
import pandas as pd 
import random
from tqdm import tqdm

# 자세한 error출력을 위한 코드
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

import cv2

#################################################################
# detr github clone code
import sys
sys.path.append('./detr/')

from detr.models.matcher import HungarianMatcher
from detr.models.detr import SetCriterion
#################################################################

import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.13 (you have 1.4.12). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


In [2]:
class AverageMeter(object):
    """평균 및 현재 값을 계산하고 저장합니다"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [3]:
# 각 class별 name 개수
house_nc = 5
tree_nc = 6
person_nc = 5

In [4]:
# 기본값 설정
seed = 42
num_classes = 6
num_queries = 100
null_class_coef = 0.5
BATCH_SIZE = 4
LR = 2e-5
EPOCHS = 1000

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [6]:
seed_everything(seed)

In [8]:
train_dir = "/home/connet/jay_deb/DMC-Connet-Team2-Project-1/cutmix/cutmix_images/tree/train"
val_dir = "/home/connet/jay_deb/DMC-Connet-Team2-Project-1/cutmix/cutmix_images/tree/valid"

# train, val df데이터셋을 불러오는 함수
def get_df(path):
    df = pd.DataFrame(columns=['image_id', 'width', 'height', 'name', 'x', 'y', 'w', 'h'])
    file_names = os.listdir(path +'/labels')
    i = 0
    for file_name in file_names:
        image_id = file_name.rsplit('.', 1)[0]
        width, height, _ = cv2.imread(f"{path + '/images'}/{image_id}.jpg").shape
        with open(f'{path + "/labels"}/{file_name}', 'r') as f:
            labels = [line.strip().split() for line in f.readlines()]
        for label in labels:
            name, xc, yc, w, h = label
            df.loc[i, 'image_id'] = image_id
            df.loc[i, 'width'] = width
            df.loc[i, 'height'] = height
            df.loc[i, 'name'] = name
            df.loc[i, 'x'] = float(xc) - (float(w) / 2)
            df.loc[i, 'y'] = float(yc) - (float(h) / 2)
            df.loc[i, 'w'] = float(w)
            df.loc[i, 'h'] = float(h)
            i += 1
    return df

train_df = get_df(train_dir)
val_df = get_df(val_dir)

train_df

,image_id,width,height,name,x,y,w,h
0,cutmix_216,512,512,5,0.16748,0.017578,0.750977,0.980469
1,cutmix_216,512,512,1,0.271484,0.0,0.552734,0.199219
2,cutmix_216,512,512,2,0.166992,0.202148,0.203125,0.216797
3,cutmix_216,512,512,3,0.687988,0.23877,0.229492,0.225586
4,cutmix_216,512,512,3,0.488281,0.26416,0.177734,0.297852
...,...,...,...,...,...,...,...,...
18827,5_28_23079_tree_jpg.rf.1b64a537a1fa1f220d80315...,512,512,5,0.222656,0.099609,0.378906,0.894531
18828,5_28_23079_tree_jpg.rf.1b64a537a1fa1f220d80315...,512,512,1,0.214355,0.092773,0.395508,0.478516
18829,5_28_23079_tree_jpg.rf.1b64a537a1fa1f220d80315...,512,512,0,0.294922,0.212891,0.248047,0.238281
18830,cutmix_993,512,512,5,0.308594,0.087891,0.234375,0.599609


In [9]:
# 형 일치 함수
for col in ['x', 'y', 'w', 'h']:
    train_df[col] = train_df[col].astype(float)
    val_df[col] = val_df[col].astype(float)

In [10]:
# DETR을 위한 CustomDataset
class CustomDataset(Dataset):
    def __init__(self,path,dataframe,transforms=None):
        self.path = path
        self.image_ids = os.listdir(path)
        self.df = dataframe
        self.transforms = transforms
        
    def __len__(self) -> int:
        return len(self.image_ids)
    
    def __getitem__(self,index):
        image_id = self.image_ids[index].rsplit('.', 1)[0]
        records = self.df[self.df['image_id'] == image_id]
        
        image = cv2.imread(f'{self.path}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        # image /= 255.0
        
        # DETR은 coco 형식의 데이터를 가져옵니다. 
        boxes = records[['x', 'y', 'w', 'h']].values
        
        # boundary box의 넓이
        area = boxes[:,2]*boxes[:,3]
        area = torch.as_tensor(area, dtype=torch.int32)

        labels =  np.array(records[['name']].values, dtype=np.int32).reshape(-1)
        
        if self.transforms:
            sample = {
                'image': image,
                'bboxes': boxes,
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            boxes = sample['bboxes']
            labels = sample['labels']
        
        boxes = sample['bboxes']
        target = {}
        target['boxes'] = torch.as_tensor(boxes,dtype=torch.float32)
        target['labels'] = torch.as_tensor(labels,dtype=torch.long)
        target['image_id'] = torch.tensor([index])
        target['area'] = area
        
        
        return image, target, image_id

In [11]:
def get_train_transforms():
    return A.Compose([ToTensorV2(p=1.0)],
                      bbox_params=A.BboxParams(format='coco',min_area=0, min_visibility=0,label_fields=['labels'])
                      )

def get_valid_transforms():
    return A.Compose([ToTensorV2(p=1.0)], 
                      bbox_params=A.BboxParams(format='coco',min_area=0, min_visibility=0,label_fields=['labels'])
                      )

In [13]:
train_Dset = CustomDataset(train_dir + '/images',train_df)
val_Dset = CustomDataset(val_dir + '/images',val_df)

In [14]:
class DETRModel(nn.Module):
    def __init__(self,num_classes,num_queries):
        super(DETRModel,self).__init__()
        self.num_classes = num_classes
        self.num_queries = num_queries
        
        self.model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)
        self.in_features = self.model.class_embed.in_features
        
        self.model.class_embed = nn.Linear(in_features=self.in_features,out_features=self.num_classes)
        self.model.num_queries = self.num_queries
        
    def forward(self,images):
        return self.model(images)

In [15]:
matcher = HungarianMatcher()

weight_dict = {'loss_ce': 1, 'loss_bbox': 1 , 'loss_giou': 1}

losses = ['labels', 'boxes', 'cardinality']

In [16]:
def train_fn(data_loader,model,criterion,optimizer,device,scheduler,epoch):
    model.train()
    criterion.train()
    
    summary_loss = AverageMeter()
    
    tk0 = tqdm(data_loader, total=len(data_loader))
    
    for step, (images, targets, image_ids) in enumerate(tk0):
        
        images = [image.to(device) for image in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        output = model(images)
        
        loss_dict = criterion(output, targets)
        weight_dict = criterion.weight_dict
        
        
        losses = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)
        
        optimizer.zero_grad()

        losses.backward()
        optimizer.step()
        
        summary_loss.update(losses.item(),BATCH_SIZE)
        tk0.set_postfix(loss=summary_loss.avg)
        
    return summary_loss

In [17]:
def eval_fn(data_loader, model,criterion, device):
    model.eval()
    criterion.eval()
    summary_loss = AverageMeter()
    
    with torch.no_grad():
        
        tk0 = tqdm(data_loader, total=len(data_loader))
        for step, (images, targets, image_ids) in enumerate(tk0):
            
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            output = model(images)
        
            loss_dict = criterion(output, targets)
            weight_dict = criterion.weight_dict
        
            losses = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)
            
            summary_loss.update(losses.item(),BATCH_SIZE)
            tk0.set_postfix(loss=summary_loss.avg)
    
    return summary_loss

In [18]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [20]:
def run():

    train_dataset = CustomDataset(
    train_dir + '/images',
    train_df,
    transforms=get_train_transforms()
    )

    valid_dataset = CustomDataset(
    val_dir + '/images',
    val_df,
    transforms=get_valid_transforms()
    )
    
    train_data_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn
    )

    valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
    )
    
    device = torch.device('cuda')
    model = DETRModel(num_classes=num_classes,num_queries=num_queries)
    model.load_state_dict(torch.load('./bestpt/tree_detr_best_6.pth'))
    model = model.to(device)
    criterion = SetCriterion(num_classes-1, matcher, weight_dict, eos_coef = null_class_coef, losses=losses)
    criterion = criterion.to(device)
    

    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
    scheduler = CosineAnnealingLR(optimizer, T_max=50)
    best_loss = 10**5
    for epoch in range(EPOCHS):
        train_loss = train_fn(train_data_loader, model,criterion, optimizer,device,epoch=epoch, scheduler=None)
        valid_loss = eval_fn(valid_data_loader, model,criterion, device)
        
        print('|EPOCH {}| TRAIN_LOSS {}| VALID_LOSS {}|'.format(epoch+1,train_loss.avg,valid_loss.avg))
        
        if valid_loss.avg < best_loss:
            best_loss = valid_loss.avg
            print('Best model found for Epoch {}........Saving Model'.format(epoch+1))
            torch.save(model.state_dict(), f'detr_best_{epoch}.pth')
        scheduler.step()

In [21]:
run()

/home/connet/anaconda3/envs/team2_proj1/lib/python3.9/site-packages/albumentations/core/composition.py:161: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()
Using cache found in /home/connet/.cache/torch/hub/facebookresearch_detr_main
/home/connet/anaconda3/envs/team2_proj1/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/connet/anaconda3/envs/team2_proj1/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_1960807/3566605769.py:33: Futu

KeyboardInterrupt: 

In [ ]:
test_dir = "../data/roboflow/person/test"
test_df = get_df(test_dir)

for col in ['x', 'y', 'w', 'h']:
    test_df[col] = test_df[col].astype(float)

test_df

,image_id,width,height,name,x,y,w,h
0,3_11_23066_person_jpg.rf.81e5f8a5c40558e49edb1...,512,512,4,0.488281,0.152344,0.300781,0.738281
1,3_11_23066_person_jpg.rf.81e5f8a5c40558e49edb1...,512,512,1,0.585938,0.234375,0.037109,0.050781
2,3_11_23066_person_jpg.rf.81e5f8a5c40558e49edb1...,512,512,2,0.609375,0.453125,0.125000,0.394531
3,3_11_23066_person_jpg.rf.81e5f8a5c40558e49edb1...,512,512,2,0.517578,0.478516,0.097656,0.394531
4,3_11_23066_person_jpg.rf.81e5f8a5c40558e49edb1...,512,512,1,0.617188,0.220703,0.042969,0.050781
...,...,...,...,...,...,...,...,...
729,3_14_23008_person_jpg.rf.e35a32bace688fbf8af61...,512,512,1,0.476562,0.066406,0.050781,0.087891
730,3_14_23008_person_jpg.rf.e35a32bace688fbf8af61...,512,512,0,0.336914,0.193848,0.123047,0.166992
731,3_14_23008_person_jpg.rf.e35a32bace688fbf8af61...,512,512,2,0.398438,0.228516,0.076172,0.318359
732,3_14_23008_person_jpg.rf.e35a32bace688fbf8af61...,512,512,2,0.464355,0.190430,0.106445,0.384766


In [ ]:
def view_sample(df_test,model,device):
    test_dataset = CustomDataset(path=test_dir+'/images',
                                 dataframe=df_test,
                                 transforms=get_valid_transforms()
                                )
     
    test_data_loader = DataLoader(
                                    test_dataset,
                                    batch_size=BATCH_SIZE,
                                    shuffle=False,
                                   num_workers=4,
                                   collate_fn=collate_fn)

    images, targets, image_ids = next(iter(test_data_loader))
    
    images = list(img.to(device) for img in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

    boxes = targets[0]['boxes'].cpu().numpy()
    boxes = [np.array(box* 512).astype(np.int32) for box in boxes ]
    sample = images[0].permute(1,2,0).cpu().numpy()

    model.eval()
    model.to(device)
    cpu_device = torch.device("cpu")
    
    with torch.no_grad():
        outputs = model(images)
        
    outputs = [{k: v.to(cpu_device) for k, v in outputs.items()}]
    fig, ax = plt.subplots(1, 1, figsize=(16, 8))

    oboxes = outputs[0]['pred_boxes'][0].detach().cpu().numpy()
    oboxes = [np.array(box).astype(np.int32) for box in oboxes * 512]
    probs = F.softmax(outputs[0]['pred_logits'], -1)
    max_probs, labels = probs.max(-1)

    for box, p, label in zip(oboxes, max_probs, labels):
        print(p)
        print(label)
        for i in range(len(p)):
            if p[i].item() > 0.15:  # 임계값 0.15 이상인 예측만 표시
                print(box[0], box[1], box[2], box[3])
                color = (0, 0, 220)
                cv2.rectangle(sample,
                      (box[0], box[1]),
                      (box[2], box[3]),
                      color, 1)
                cv2.putText(sample, str(label[i].item()), (box[0], box[1] - 5), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
    
    ax.set_axis_off()
    sample = sample.astype(np.uint8)
    ax.imshow(sample)

In [ ]:
model = DETRModel(num_classes=num_classes,num_queries=num_queries)
models = os.listdir('./')
models = [i for i in models if 'detr_best' in i]
best_model = models[-1]
model.load_state_dict(torch.load(best_model))
view_sample(test_df,model=model,device=torch.device('cuda'))